# 네이버 영화 140자평 리뷰 자료 스크레이핑 프로토타입


네이버 영화 웹 서비스의 평점과 140자평 데이터를 스크레이핑하여 데이터 셋으로 만드는 예제입니다.

- 네티즌 평점 / 140자평 : https://movie.naver.com/movie/point/af/list.nhn?target=after


step 1. 평점 개수 확인

>#old_content > h5 > div > strong

step 2. 데이터셋 생성

step 3. 데이터 저장

가끔 한글이 깨지는 경우가 있습니다. 아래 두줄의 코드를 넣어줍니다.

In [1]:
#! /usr/bin/python3
# -*- coding: utf-8 -*-

**라이브러리 import**

In [2]:
import numpy as np
import pandas as pd
 
import urllib.request as req
import re # 정규식 관련 라이브러리
from bs4 import BeautifulSoup # html 파싱용 라이브러리

In [3]:
BASE_URL = "https://movie.naver.com/movie/point/af/list.nhn?target=after"
PAGE_URL = BASE_URL + "?&page=%s"

**리뷰 페이지가 몇개인지 먼저 확인합니다.**

In [4]:
html = req.urlopen(BASE_URL)
soup = BeautifulSoup(html, "html.parser")

review_count = soup.select("#old_content > h5 > div > strong")
review_count = int(review_count[0].string)

review_count

11277547

In [5]:
page_count = int(np.ceil(review_count / 10)) # 한 페이지당 10개의 게시글
page_count

1127755

**테스트용 1페이지 파싱**

In [6]:
page_num = 1
URL = PAGE_URL % page_num
URL

'https://movie.naver.com/movie/point/af/list.nhn?target=after?&page=1'

In [7]:
res = req.urlopen(URL)
soup = BeautifulSoup(res, "html.parser")

In [8]:
test_tr = soup.find_all('tr')

**첫번째 행은 Header 정보입니다.**

In [21]:
test_tr[:2]

[<tr>
 <th>번호</th>
 <th colspan="2">평점</th>
 <th>140자평</th>
 <th class="al"><span class="th_m1">글쓴이·날짜</span></th>
 </tr>, <tr>
 <td class="ac num">14777316</td>
 <td><div class="fr point_type_n"><div class="mask" style="width:100.0%"></div></div></td>
 <td class="point">10</td>
 <td class="title">
 <a class="movie" href="?st=mcode&amp;sword=168050&amp;target=after">스타 이즈 본</a>
 <br/>이 영화에서야말로...진짜 스타 탄생!!! 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="javascript:report('lees****', 'qfddv0dS0RS/fee4R9E8fX1QKUk1t502uV8m6+dqpeE=', '이 영화에서야말로...진짜 스타 탄생!!!', '14777316', 'point_after');" style="color:#8F8F8F">신고</a>
 </td>
 <td class="num"><a class="author" href="javascript:find_list('nickname','14777316','after');">lees****</a><br/>18.10.26</td>
 </tr>]

## 여기서 필요한 것만 파싱합니다.

1. 리뷰 번호
2. 리뷰 평점
3. 영화 제목
4. 영화 리뷰

### 1. 리뷰 번호

In [22]:
test_num = test_tr[1].find('td', {'class':'ac num'})
test_num = test_num.text
test_num

'14777316'

### 2. 리뷰 평점

In [23]:
test_point = test_tr[1].find('td', {'class':'point'})
test_point = test_point.text
test_point

'10'

### 3. 영화 제목

In [24]:
test_movie_info = test_tr[1].find('td', {'class':'title'})
test_movie_info = test_movie_info.text
test_movie_info

'\n스타 이즈 본\n이 영화에서야말로...진짜 스타 탄생!!! \r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t신고\n'

In [25]:
movielist = test_movie_info.split('\n')

In [26]:
test_title = movielist[1].strip()
test_title

'스타 이즈 본'

### 4. 영화 리뷰

In [27]:
test_review = movielist[2].strip()
test_review

'이 영화에서야말로...진짜 스타 탄생!!!'

## 파싱 함수 만들기

위의 내용을 토대로 파싱 함수를 만듭니다.

In [28]:
def parse_item(item):
    # return 리뷰번호, 평점, 영화제목, 영화리뷰
    # num, point, title, review
    
    _num = item.find('td', {'class':'ac num'})
    _num = _num.text

    _point = item.find('td', {'class':'point'})
    _point = _point.text
    
    movie_info = item.find('td', {'class':'title'})
    movie_info = movie_info.text
    movie_info = movie_info.split('\n')
    
    _title = movie_info[1].strip()
    _review = movie_info[2].strip()
          
    return {'num': _num, 'point': _point, 'title': _title, 'review': _review}

In [29]:
def scraping_rating_page(url):   
    html = req.urlopen(url)
    soup = BeautifulSoup(html, 'html')
    
    items = soup.findAll('tr')
    items = items[1:] # 첫번째 행은 header 이므로 제거합니다.

    return list(filter(None, [parse_item(item) for item in items]))

In [32]:
scraping_rating_page(URL)

[{'num': '14777326',
  'point': '10',
  'title': '호밀밭의 반항아',
  'review': '아집불통 성실은, 고집불통 재능과 권력을 이긴다.'},
 {'num': '14777325',
  'point': '8',
  'title': '창궐',
  'review': '기대안하고 봤는데  잼있었어요~현빈 액션씬넘나 멋짐~장동건 악역연기도 굿~~'},
 {'num': '14777324',
  'point': '1',
  'title': '창궐',
  'review': '알바들한테 속는 흑우들 없제?'},
 {'num': '14777323',
  'point': '10',
  'title': '창궐',
  'review': '이영화~볼만하네요 현빈최고 입니다'},
 {'num': '14777322',
  'point': '10',
  'title': '창궐',
  'review': '장동건이 용복입고 나왔을때 포스 ㄷㄷ'},
 {'num': '14777321',
  'point': '7',
  'title': '창궐',
  'review': '오락성으로 충분히 볼만한 영화다 그리고 웹툰 원작에 충실하게 만들었네 무조건 10점 알바도 있지만 그렇다고 1점 테러는 뭐냐;; 네이버는 아이디당 1달에 한번만 평점 할수 있게 만들어라'},
 {'num': '14777320',
  'point': '1',
  'title': '창궐',
  'review': '엑스트라들이 웃으면서 전투참여하더라 ㅋㅋㅋ웃겨라. .'},
 {'num': '14777319',
  'point': '8',
  'title': '인크레더블 2',
  'review': '너무나 잼있는 가족히어로물~ 대사도 촌철살인~  가족같에 있을 수 있는 일들을 참 리얼하게 묘사했다^^'},
 {'num': '14777317', 'point': '10', 'title': '미쓰백', 'review': '으흐어어어어엉 으으윽흑흑'},
 {'num': '14777316',
  'po

**10개의 리뷰 정보가 저장되었습니다. (1페이지 분량)**

In [33]:
test_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))

for index in range(10):
    test_df = test_df.append(scraping_rating_page(PAGE_URL % (index+1)), ignore_index=True)
    
test_df

,num,point,title,review
0,14777328,4,안시성,전투신이 많아 찍을때는 힘들었을 것 같지만...대부분의 배우가 사극의 역할에는 어울...
1,14777327,10,펭귄 하이웨이,"작은 언덕같은 가슴, 포근한 가슴... 가슴 좋아하는게 뭐가 나쁨? 가슴배구단 보면..."
2,14777326,10,호밀밭의 반항아,"아집불통 성실은, 고집불통 재능과 권력을 이긴다."
3,14777325,8,창궐,기대안하고 봤는데 잼있었어요~현빈 액션씬넘나 멋짐~장동건 악역연기도 굿~~
4,14777324,1,창궐,알바들한테 속는 흑우들 없제?
5,14777323,10,창궐,이영화~볼만하네요 현빈최고 입니다
6,14777322,10,창궐,장동건이 용복입고 나왔을때 포스 ㄷㄷ
7,14777321,7,창궐,오락성으로 충분히 볼만한 영화다 그리고 웹툰 원작에 충실하게 만들었네 무조건 10점...
8,14777320,1,창궐,엑스트라들이 웃으면서 전투참여하더라 ㅋㅋㅋ웃겨라. .
9,14777319,8,인크레더블 2,너무나 잼있는 가족히어로물~ 대사도 촌철살인~ 가족같에 있을 수 있는 일들을 참 ...


- 리뷰 갯수가 딱 맞지 않을 수 있습니다.
- 이유는 스크레핑 하는 과정에도 새로운 게시물이 생겨서 페이징 시에 누락되는 평점이 존재 하기 때문입니다.**